- ref https://www.codementor.io/spark/tutorial/mllib-basic-statistics-exploratory-data-analysis
- ref http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names

In [18]:
sc

In [19]:
from pyspark.sql import SQLContext
import datetime as dt   
import time
import pandas as pd, numpy as np
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
%pylab inline
import urllib

Populating the interactive namespace from numpy and matplotlib


//anaconda/envs/g_dash/lib/python3.4/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [20]:
sqlContext = SQLContext(sc)

# import data 

In [21]:
# ref http://stackoverflow.com/questions/17960942/attributeerror-module-object-has-no-attribute-urlretrieve

f = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

#  make RDD

In [22]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [23]:
type(raw_data)

pyspark.rdd.RDD

In [24]:
raw_data.take(5)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In [25]:
# map 

csv_data = raw_data.map(lambda x: x.split(","))

In [26]:
csv_data.take(1)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.']]

In [27]:
# flatmap 
raw_data.flatMap(lambda x: x.split(",")).take(5)

['0', 'tcp', 'http', 'SF', '181']

In [28]:
csv_data.map(lambda word : (word,1)).take(5)#.reduceByKey(lambda x,y : x+y).take(4)

[(['0',
   'tcp',
   'http',
   'SF',
   '181',
   '5450',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '8',
   '8',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '1.00',
   '0.00',
   '0.00',
   '9',
   '9',
   '1.00',
   '0.00',
   '0.11',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   'normal.'],
  1),
 (['0',
   'tcp',
   'http',
   'SF',
   '239',
   '486',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '8',
   '8',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '1.00',
   '0.00',
   '0.00',
   '19',
   '19',
   '1.00',
   '0.00',
   '0.05',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   'normal.'],
  1),
 (['0',
   'tcp',
   'http',
   'SF',
   '235',
   '1337',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
  

#  Map & ReduceByKey

In [29]:
key_value_duration = csv_data.map(lambda x: (x[41], float(x[0]))) 
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)

durations_by_key.collect()

[('portsweep.', 1991911.0),
 ('neptune.', 0.0),
 ('satan.', 64.0),
 ('pod.', 0.0),
 ('multihop.', 1288.0),
 ('back.', 284.0),
 ('warezclient.', 627563.0),
 ('nmap.', 0.0),
 ('smurf.', 0.0),
 ('guess_passwd.', 144.0),
 ('ftp_write.', 259.0),
 ('imap.', 72.0),
 ('land.', 0.0),
 ('loadmodule.', 326.0),
 ('buffer_overflow.', 2751.0),
 ('perl.', 124.0),
 ('ipsweep.', 43.0),
 ('rootkit.', 1008.0),
 ('phf.', 18.0),
 ('teardrop.', 0.0),
 ('warezmaster.', 301.0),
 ('normal.', 21075991.0),
 ('spy.', 636.0)]

In [31]:
key_value_duration = csv_data.map(lambda x: (x[2], float(x[0]))) 
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)

durations_by_key.collect()

[('finger', 737.0),
 ('http', 81411.0),
 ('netbios_dgm', 0.0),
 ('name', 0.0),
 ('hostnames', 0.0),
 ('vmnet', 0.0),
 ('systat', 0.0),
 ('shell', 0.0),
 ('netbios_ssn', 0.0),
 ('urh_i', 0.0),
 ('pop_3', 40444.0),
 ('ctf', 0.0),
 ('domain', 6.0),
 ('mtp', 0.0),
 ('remote_job', 0.0),
 ('exec', 0.0),
 ('supdup', 40686.0),
 ('http_443', 0.0),
 ('sunrpc', 11.0),
 ('urp_i', 0.0),
 ('pop_2', 0.0),
 ('csnet_ns', 40452.0),
 ('smtp', 27626.0),
 ('whois', 4.0),
 ('ldap', 0.0),
 ('daytime', 0.0),
 ('imap4', 72.0),
 ('nntp', 5.0),
 ('klogin', 0.0),
 ('rje', 1.0),
 ('IRC', 315753.0),
 ('link', 0.0),
 ('eco_i', 0.0),
 ('tftp_u', 0.0),
 ('iso_tsap', 0.0),
 ('uucp_path', 4.0),
 ('auth', 88.0),
 ('ecr_i', 0.0),
 ('other', 20379049.0),
 ('domain_u', 124372.0),
 ('courier', 30619.0),
 ('discard', 11.0),
 ('red_i', 0.0),
 ('tim_i', 0.0),
 ('time', 25.0),
 ('login', 201.0),
 ('ftp', 337665.0),
 ('telnet', 456677.0),
 ('ntp_u', 1889.0),
 ('sql_net', 0.0),
 ('echo', 0.0),
 ('private', 1398901.0),
 ('gopher', 